In [1]:
import torch
from torch.autograd import Function, Variable
import torch.nn.functional as F
from torch import nn
from torch.nn.parameter import Parameter
import torch.optim as optim
from torch.nn.utils import parameters_to_vector
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import numpy.random as npr

import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt

# Comment these lines if your MPC is in the current directory.
# Otherwise modify to the directory.
import sys
sys.path.append('./../mpc.pytorch/')


from mpc import mpc
from mpc.mpc import GradMethods, QuadCost, LinDx
from mpc.dynamics import NNDynamics
import mpc.util as eutil
from mpc.env_dx import pendulum, cartpole, frenet_kin_bicycle
from mpc.track.src import simple_track_generator, track_functions


#import sys
#from IPython.core import ultratb
#sys.excepthook = ultratb.FormattedTB(mode='Verbose',
#     color_scheme='Linux', call_pdb=1)

import time
import os
import shutil
import pickle as pkl
import collections

import argparse

In [2]:
# Let's try to create a track 

track_density = 300
track_width = 0.5
gen = simple_track_generator.trackGenerator(track_density,track_width)
track_name = 'DEMO_TRACK'

In [3]:
t = 0.3
init = [0,0,0]

track_function = {
    'DEMO_TRACK'    : track_functions.demo_track,
    'HARD_TRACK'    : track_functions.hard_track,
    'LONG_TRACK'    : track_functions.long_track,
    'LUCERNE_TRACK' : track_functions.lucerne_track,
    'BERN_TRACK'    : track_functions.bern_track,
    'INFINITY_TRACK': track_functions.infinity_track,
    'SNAIL_TRACK'   : track_functions.snail_track
}.get(track_name, track_functions.demo_track)
    
track_function(gen, t, init)
    
gen.populatePointsAndArcLength()
gen.centerTrack()

track_coord = torch.from_numpy(np.vstack([gen.xCoords, gen.yCoords, gen.arcLength, gen.tangentAngle, gen.curvature]))

In [4]:
device = 'cpu'
softplus_op = torch.nn.Softplus(10)

def sample_xinit(n_batch):
    def uniform(shape, low, high):
        r = high-low
        return torch.rand(shape)*r+low

    sigma = uniform(n_batch, 0.01, 2.)
    d = uniform(n_batch, -0.1, 0.1)
    phi = uniform(n_batch, -0.40*np.pi, 0.40*np.pi)
    v = uniform(n_batch, 0., 0.2)
    d_lb = softplus_op(-d-0.5*track_width)
    d_ub = softplus_op(d-0.5*track_width)
    xinit = torch.stack((sigma, d, phi, v, d_lb, d_ub), dim=1)

    return xinit

true_dx = frenet_kin_bicycle.FrenetKinBicycleDx(track_coord)
mpc_T = 15
n_batch = 8

# Added here the bounds of U
u_lower = torch.tensor([-2., -1.]).unsqueeze(0).unsqueeze(0).repeat(mpc_T, n_batch, 1)
u_upper = torch.tensor([2., 1.]).unsqueeze(0).unsqueeze(0).repeat(mpc_T, n_batch, 1)

n_state = true_dx.n_state
n_ctrl = true_dx.n_ctrl

u_init=None
eps = 1
lqr_iter = 500
grad_method = GradMethods.AUTO_DIFF

In [5]:
# each mpc returns a prediction of the states and the inputs over the horizon for all batches and all dimensions,
# they are stored in the following order pred_x = [horizon,batch, dimension].
# To penalize a control behaviour that does not enforce enough progress within a horizon we can try to chose 
# our loss as -pred_x[mpc_T-1,:,0]

def get_loss_progress(x_init, dx, _Q, _p, mpc_T=mpc_T):    
        
        pred_x, pred_u, pred_objs = mpc.MPC(
            dx.n_state, dx.n_ctrl, mpc_T,
            u_lower=u_lower, u_upper=u_upper, u_init=u_init,
            lqr_iter=lqr_iter,
            verbose=0,
            exit_unconverged=False,
            detach_unconverged=True,
            linesearch_decay=dx.linesearch_decay,
            max_linesearch_iter=dx.max_linesearch_iter,
            grad_method=grad_method,
            eps=0.5,
            n_batch=n_batch,
        )(x_init, QuadCost(_Q, _p), dx)
        
        #I added the second term to account the initial state (to kind of normalize the progress)
        progress_loss = torch.mean(-pred_x[mpc_T-1,:,0] + pred_x[0,:,0])
        
        #penalty_loss = pred_x[]
            
        return progress_loss

In [6]:
params1 = []
learn_q_logit_state = torch.ones(n_state-2, requires_grad=True).to(device)
learn_q_logit_input = torch.ones(n_ctrl, requires_grad=True).to(device)
learn_p_state = torch.zeros(n_state-2, requires_grad=True).to(device)
learn_p_input = torch.zeros(n_ctrl, requires_grad=True).to(device)
params1 += [learn_q_logit_state, learn_q_logit_input, learn_p_state, learn_p_input]
env_params = true_dx.params
q_penalty = .01*torch.ones(2).to(device)
p_penalty = 10*torch.ones(2).to(device)
track_coord = track_coord.to(device)

In [7]:
#learn_q_logit = torch.ones_like(true_q, requires_grad=True).to(device)
#learn_p = torch.zeros_like(true_p, requires_grad=True).to(device)

In [8]:
params = [{
            'params': params1,
            'lr': 2e-3,
            'alpha': 0.99,
            }]
dx = true_dx.__class__(track_coord,env_params)

opt = optim.RMSprop(params)

In [9]:
for i in range(150):

    q = torch.cat((learn_q_logit_state,q_penalty,learn_q_logit_input),dim=0)
    p = torch.cat((learn_p_state,p_penalty,learn_p_input), dim=0)
    #print(q)

    Q_batch = torch.diag(q).unsqueeze(0).unsqueeze(0).repeat(
                mpc_T, n_batch, 1, 1
            )
    p_batch = p.unsqueeze(0).repeat(mpc_T, n_batch, 1)

    x_init = sample_xinit(n_batch).to(device)
    #im_loss = get_loss_cost(x_init, dx, Q_batch, p_batch)
    im_loss = get_loss_progress(x_init, dx, Q_batch, p_batch)

    opt.zero_grad()
    im_loss.backward()
    opt.step()
    
    print('Batch:', i , ' Progression with MPC_T=',mpc_T ,': ', -round(im_loss.item(), 4))

Batch: 0  Progression with MPC_T= 15 :  0.0413
Batch: 1  Progression with MPC_T= 15 :  0.0349
Batch: 2  Progression with MPC_T= 15 :  0.0625
Batch: 3  Progression with MPC_T= 15 :  0.0548
Batch: 4  Progression with MPC_T= 15 :  0.065
Batch: 5  Progression with MPC_T= 15 :  0.0396
Batch: 6  Progression with MPC_T= 15 :  0.0786
Batch: 7  Progression with MPC_T= 15 :  0.0419
Batch: 8  Progression with MPC_T= 15 :  0.0449
Batch: 9  Progression with MPC_T= 15 :  0.0473
Batch: 10  Progression with MPC_T= 15 :  0.0487
Batch: 11  Progression with MPC_T= 15 :  0.0742
Batch: 12  Progression with MPC_T= 15 :  0.0273
Batch: 13  Progression with MPC_T= 15 :  0.0736
Batch: 14  Progression with MPC_T= 15 :  0.0711
Batch: 15  Progression with MPC_T= 15 :  0.0553
Batch: 16  Progression with MPC_T= 15 :  0.0877
Batch: 17  Progression with MPC_T= 15 :  0.0829
Batch: 18  Progression with MPC_T= 15 :  0.0567
Batch: 19  Progression with MPC_T= 15 :  0.0859
Batch: 20  Progression with MPC_T= 15 :  0.0922
Bat

KeyboardInterrupt: 

In [10]:
# Here we can choose the number of samples we want to test (number of initial states)
N_test = 1

# Whatever I wrote below might be wrong, we have to see if we really can change the mpc_T to Test, 
# it gives weird results sometimes.
# Here we can choose the mpc_T in the test time, which can be much higher than in the training.
# Ideally, we would like to have the whole lap here, I guess. But we need to fix the warnings/errors before.
mpc_T_test = 30

In [11]:
x_init_test = sample_xinit(N_test)

In [12]:
# Below you can put any initial state you want (any that make sense)
x_init_test = torch.tensor([[1.6, 0.1, -0.8, 0.1,softplus_op(torch.Tensor([-0.1-0.5*track_width])), softplus_op(torch.Tensor([-0.1-0.5*track_width]))]])

In [13]:
Q_test = torch.diag(q).unsqueeze(0).unsqueeze(0).repeat(
                mpc_T_test, N_test, 1, 1
            )
p_test = p.unsqueeze(0).repeat(mpc_T_test, N_test, 1)

In [14]:
true_q, true_p = true_dx.get_true_obj()
true_q = torch.Tensor([ 0.,  3.,  1.,  0., 0., 0., 1., 2.])
true_p = torch.Tensor([ -2.,  0.,  0.,  0., 100., 100.,  -1,  0.])

true_q = true_q.to(device)
true_p = true_p.to(device)

expert_Q = torch.diag(true_q).unsqueeze(0).unsqueeze(0).repeat(
            mpc_T_test, N_test, 1, 1
        )
expert_p = true_p.unsqueeze(0).repeat(mpc_T_test, N_test, 1)

In [15]:
# Added here the bounds of U
u_lower_test = torch.tensor([-2., -1.]).unsqueeze(0).unsqueeze(0).repeat(mpc_T_test, N_test, 1)
u_upper_test = torch.tensor([2., 1.]).unsqueeze(0).unsqueeze(0).repeat(mpc_T_test, N_test, 1)

In [16]:
x_mpc_test, u_mpc_test, objs_mpc_test = mpc.MPC(
            n_state, n_ctrl, mpc_T_test,
            u_lower=u_lower_test, u_upper=u_upper_test, u_init=u_init,
            lqr_iter=lqr_iter,
            verbose=0,
            exit_unconverged=False,
            detach_unconverged=True,
            linesearch_decay=dx.linesearch_decay,
            max_linesearch_iter=dx.max_linesearch_iter,
            grad_method=grad_method,
            eps=eps,
            n_batch=N_test,
        )(x_init_test, QuadCost(expert_Q, expert_p), dx)

[WARNING] pnqp warning: Did not converge
[WARNING] pnqp warning: Did not converge
[WARNING] pnqp warning: Did not converge
[WARNING] pnqp warning: Did not converge
[WARNING] pnqp warning: Did not converge
LQR Warning: All examples did not converge to a fixed point.
Detaching and *not* backpropping through the bad examples.


In [17]:
def frenet_to_cartesian(point_f, ref_path):
    
    def get_nearest_index(point_f, ref_path):
        return ((point_f[0] - ref_path[2,:])**2).argmin()
    
    def compute_x_coord(point_f, ref_path, nearest_index):
        return ref_path[0,nearest_index] - point_f[1]*torch.sin(ref_path[3,nearest_index])
    
    def compute_y_coord(point_f, ref_path, nearest_index):
        return ref_path[1,nearest_index] + point_f[1]*torch.cos(ref_path[3,nearest_index])
    
    nearest_index = get_nearest_index(point_f, ref_path)
    x = compute_x_coord(point_f, ref_path, nearest_index)
    y = compute_y_coord(point_f, ref_path, nearest_index)
    
    return torch.tensor([x, y])

In [18]:
x_list = []
y_list = []

for i in range(mpc_T_test):
    xy = frenet_to_cartesian(x_mpc_test[i,0,:2], track_coord)
    x_list.append(xy[0].numpy())
    y_list.append(xy[1].numpy())

In [19]:
x_plot = np.array(x_list)
y_plot = np.array(y_list)

In [20]:
u_mpc_test.max(0)[0].max(0)[0]

tensor([1.6193, 0.8438], grad_fn=<MaxBackward0>)

In [21]:
fig, ax = plt.subplots(1,1, figsize=(10,5), dpi=120)
gen.plotPoints(ax)
#gen.pointAtArcLength(0)
#gen.writePointsToYaml('../tracks/' + track_name + '.yaml', track_density)

ax.scatter(x_plot, y_plot, s=4, color='red')

print('x_init: ' + str(gen.xCoords[0]))
print('y_init: ' + str(gen.yCoords[0]))
print('yaw_init: ' + str(gen.tangentAngle[0]))
print('Total Arc Length: ' + str(gen.arcLength[-1]/2))
plt.show()

ImportError: cannot import name 'ImageTk' from 'PIL' (/usr/lib/python3/dist-packages/PIL/__init__.py)